In [1]:
import numpy as np

import Go_Board
import time
import random
import MCTS_GO

from sgfmill import sgf

import pandas as pd

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

In [2]:
dataset_dir = './go9-large/'#dataset/sgf/train/0000/'

from os import listdir
from os.path import isfile, join
examples = [f for f in listdir(dataset_dir) if isfile(join(dataset_dir, f))]

#example = '2014-12-12T14:25:20.109Z_ls125rvjkot8.sgf'
board_size = 9
examples = examples[:3000]

In [3]:
X = []
Y = []

for example in examples:
    board_go = Go_Board.Our_Board(board_size=board_size)
    with open(dataset_dir+example, "rb") as f:
        game = sgf.Sgf_game.from_bytes(f.read())
    winner = game.get_winner()
    board_size = game.get_size()
    root_node = game.get_root()

    board = np.zeros(shape=(board_size,board_size))

    for node in game.get_main_sequence():

        if node.get_move()[0] is not None:
            move = node.get_move()
            player = move[0]
            coord = (move[1][1],move[1][0]) if move[1] is not None else (-1,-1)

            board  = board_go.get_matrix()

            y = np.zeros(shape=(board_size,board_size))
            if(coord[0] != -1 and coord[1] != -1):
                y[coord[0]][coord[1]] = 1
                y = y.reshape(board_size*board_size)
                y = np.append(y,[0])
                y = y.reshape(board_size*board_size+1)
            else:
                y = np.append(y,[1])
                y = y.reshape(board_size*board_size+1)

            x = board * -1 if player == 'w' else board
            board_go.push(coord)
            X.append(x)
            Y.append(y)

In [4]:
X = np.reshape(X,(len(X),board_size,board_size,1))

In [5]:
np.shape(X)

(144556, 9, 9, 1)

In [6]:
Y = np.reshape(Y,(len(Y),82))
Y.shape

(144556, 82)

In [7]:
#Y = tf.keras.utils.to_categorical(Y,num_classes=82)
#Y.shape

In [8]:
model = models.Sequential()

model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=(9, 9, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(82, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 8, 32)          160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
dense (Dense)                (None, 4, 4, 64)          2112      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 3, 32)          8224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8

In [9]:
model(X).shape


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



TensorShape([144556, 82])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X, Y, epochs=25,batch_size=32)


Train on 144556 samples
Epoch 1/25
144556/144556 [==============================] - 79s 547us/sample - loss: 4.0182 - accuracy: 0.0500
Epoch 2/25
144556/144556 [==============================] - 102s 709us/sample - loss: 3.8429 - accuracy: 0.0824
Epoch 3/25
 63424/144556 [============>.................] - ETA: 1:17 - loss: 3.7780 - accuracy: 0.0987